In [ ]:
import os, sys
target = "BacktesterOHLCV"
while os.path.basename(os.getcwd()) != target:
    os.chdir("..")
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

In [ ]:
from engine.dataClass import OHLCVDataLoader
from engine.feature_preprocessing import generate_flat_ml_dataframe
from strats.MAstrat import MAStrat
from strats.RSIStrat import RSIStrat
from engine.analysis_test import AssetTradeAnalysis
#from ANNStrat import ANNStrat
from strats.MomentumStratV2 import MomentumStratV2
from engine.backtester import OHLCVBacktester
from engine.analysis import OHLCVAnalysis
import os

In [ ]:
# 1. Load and clean data
folder_path = "data/stock_market_data/sp500/csv"
SP500Path = "data/SP500Historical.csv"
data = OHLCVDataLoader.load_all_data(folder_path, clean=True, years=20,sp500_path=SP500Path)

## Instiatiate strategies

In [ ]:
#2. Instiatiate strategies
ma1 = MAStrat(name="MA 1 (Short-term)", short_window=5, long_window=20) # Short-term setup
ma2 = MAStrat(name="MA 2 (Balanced)", short_window=15, long_window=45) # Balanced crossover
ma3 = MAStrat(name="MA 3 (Medium-term Swing)", short_window=10, long_window=50) # Medium-term (Swing trading)
ma4 = MAStrat(name="MA 4 (Medium-term Trend)", short_window=20, long_window=100) # Medium-term (trend following)
ma5 = MAStrat(name="MA 5 (Long-term)", short_window=50, long_window=200) # Long-term setup

rsi1 = RSIStrat(name="RSI 1 (Wilder's default)", rsi_window=14, sellThreshold=70, buyThreshold=30, alpha=1 ) 
rsi2 = RSIStrat(name="RSI 2 (Aggressive)", rsi_window=10, sellThreshold=75, buyThreshold=25, alpha=2)

hybrid1 = MomentumStratV2(name="Hybrid 1", short_window=10, long_window=50, rsi_window=14, sellThreshold=70, buyThreshold=30, alpha=1) # Classing swing + smooth RSI
hybrid2 = MomentumStratV2(name="Hybrid 2", short_window=20, long_window=100, rsi_window=10, sellThreshold=75, buyThreshold=25, alpha=2) # Medium-term + reactive RSI
hybrid3 = MomentumStratV2(name="Hybrid 3", short_window=15, long_window=45, rsi_window=14, sellThreshold=60, buyThreshold=40, alpha=1) # Balanced MA + tight RSI

## Backtesting

In [ ]:
#3. Run backtest
backtest_ma1 = OHLCVBacktester(data, ma1, fee_perc=0.0005)
backtest_ma2 = OHLCVBacktester(data, ma2, fee_perc=0.0005)
backtest_ma3 = OHLCVBacktester(data, ma3, fee_perc=0.0005)
backtest_ma4 = OHLCVBacktester(data, ma4, fee_perc=0.0005)
backtest_ma5 = OHLCVBacktester(data, ma5, fee_perc=0.0005)

backtest_rsi1 = OHLCVBacktester(data, rsi1, fee_perc=0.0005)
backtest_rsi2 = OHLCVBacktester(data, rsi2, fee_perc=0.0005)

backtest_hybrid1 = OHLCVBacktester(data, hybrid1, fee_perc=0.0005)
backtest_hybrid2 = OHLCVBacktester(data, hybrid2, fee_perc=0.0005)
backtest_hybrid3 = OHLCVBacktester(data, hybrid3, fee_perc=0.0005)


In [ ]:
states_ma1, portfolio_ma1 = backtest_ma1.run()
states_ma2, portfolio_ma2 = backtest_ma2.run()
states_ma3, portfolio_ma3 = backtest_ma3.run()
states_ma4, portfolio_ma4 = backtest_ma4.run()
states_ma5, portfolio_ma5 = backtest_ma5.run()

In [ ]:
states_rsi1, portfolio_rsi1 = backtest_rsi1.run()
states_rsi2, portfolio_rsi2 = backtest_rsi2.run()

In [ ]:
states_hybrid1, portfolio_hybrid1 = backtest_hybrid1.run()
states_hybrid2, portfolio_hybrid2 = backtest_hybrid2.run()
states_hybrid3, portfolio_hybrid3 = backtest_hybrid3.run()

In [ ]:
#4. Analysis

analysis_ma1 = OHLCVAnalysis(states_ma1, portfolio_ma1, data, strategy= ma1)
analysis_ma2 = OHLCVAnalysis(states_ma2, portfolio_ma2, data, strategy= ma2)
analysis_ma3 = OHLCVAnalysis(states_ma3, portfolio_ma3, data, strategy= ma3)
analysis_ma4 = OHLCVAnalysis(states_ma4, portfolio_ma4, data, strategy= ma4)
analysis_ma5 = OHLCVAnalysis(states_ma5, portfolio_ma5, data, strategy= ma5)

analysis_rsi1 = OHLCVAnalysis(states_rsi1, portfolio_rsi1, data, strategy= rsi1)
analysis_rsi2 = OHLCVAnalysis(states_rsi2, portfolio_rsi2, data, strategy= rsi2)

analysis_hybrid1 = OHLCVAnalysis(states_hybrid1, portfolio_hybrid1, data, strategy= hybrid1)
analysis_hybrid2 = OHLCVAnalysis(states_hybrid2, portfolio_hybrid2, data, strategy= hybrid2)
analysis_hybrid3 = OHLCVAnalysis(states_hybrid3, portfolio_hybrid3, data, strategy= hybrid3)

chaque asset
Pnl pour chaque strat


In [ ]:
analysis_new_hybrid2 = AssetTradeAnalysis(states_hybrid2, data, strategy= hybrid2)
analysis_new_hybrid2.plot_asset_trades('DXCM')
analysis_new_hybrid2.plot_asset_pnl_with_trades('DXCM')
analysis_new_hybrid2.plot_asset_position('DXCM')


In [ ]:
analysis_ma1.print_portfolio_metrics(full=True)
analysis_ma2.print_portfolio_metrics(full=True)
analysis_ma3.print_portfolio_metrics(full=True)
analysis_ma4.print_portfolio_metrics(full=True)
analysis_ma5.print_portfolio_metrics(full=True)

analysis_rsi1.print_portfolio_metrics(full=True)
analysis_rsi2.print_portfolio_metrics(full=True)

analysis_hybrid1.print_portfolio_metrics(full=True)
analysis_hybrid2.print_portfolio_metrics(full=True)
analysis_hybrid3.print_portfolio_metrics(full=True)

In [ ]:
OHLCVAnalysis.plot_multiple_portfolio_pnl([analysis_ma1,analysis_ma2,analysis_ma3,analysis_ma4,analysis_ma5,analysis_rsi1,analysis_rsi2,analysis_hybrid1,analysis_hybrid2,analysis_hybrid3])

In [ ]:
OHLCVAnalysis.plot_multiple_portfolio_pnl([analysis_ma1,analysis_ma2,analysis_ma3,analysis_ma4,analysis_ma5])
OHLCVAnalysis.plot_multiple_portfolio_pnl([analysis_rsi1,analysis_rsi2])
OHLCVAnalysis.plot_multiple_portfolio_pnl([analysis_hybrid1,analysis_hybrid2,analysis_hybrid3]) 

Best strategy is hybrid 2 with the following parameters : 
short MA = 20
long MA = 100
RSI window = 10
RSI BuyThreshold = 25
RSI SellThreshold = 75
alpha = 2

short_window=20, long_window=100, rsi_window=10, sellThreshold=75, buyThreshold=25, alpha=2

In [ ]:
analysis_hybrid2.plot_asset_pnl_with_trades('DXCM')
analysis_hybrid2.plot_asset_trades('DXCM')
analysis_hybrid2.plot_average_position_size()


## ML

In [ ]:
from ANNStrat import ANNStrat

In [ ]:
asset_list = sorted({c.split('_')[0] for c in data.columns if c.endswith("_Adjusted Close")})
features_df = generate_flat_ml_dataframe(data, asset_list=asset_list, hybrid_strategy=hybrid2)

# Check the percentage of 0 signals in features_df
zero_signals_percentage = (features_df['signal'].value_counts().get(0, 0) / len(features_df)) * 100
print(f"Percentage of 0 signals: {zero_signals_percentage:.10f}%")

In [ ]:
selected_features = ['return', 'log_return', 'volume_zscore', 'short_ma', 'long_ma', 'ma_diff', 'rsi']
# Define and train the strategy
ANN_strategy = ANNStrat(name="ANN_Hybrid",features=selected_features)

# Step 2: Load the labeled data
ANN_strategy.load_data(features_df)

# Step 3: Train the neural network
ANN_strategy.train_model()

In [ ]:
backtest_ANN = OHLCVBacktester(data, ANN_strategy)
asset_states_ann, equity_curve_ann = backtest_ANN.run()

In [ ]:
analysis_ann = OHLCVAnalysis(asset_states_ann, equity_curve_ann, data, strategy=ANN_strategy)
analysis_ann.print_portfolio_metrics(full=True)
OHLCVAnalysis.plot_multiple_portfolio_pnl([analysis_ann,analysis_hybrid2])

In [ ]:
selected_features = ['Adjusted Close', 'Previous Close', 'volume_zscore', 'short_ma', 'long_ma', 'ma_diff', 'rsi']
# Define and train the strategy
ANN2_strategy = ANNStrat(name="ANN2_Hybrid",features=selected_features)

# Step 2: Load the labeled data
ANN2_strategy.load_data(features_df)

# Step 3: Train the neural network
ANN2_strategy.train_model()

In [ ]:
backtest_ANN2 = OHLCVBacktester(data, ANN2_strategy)
asset_states_ann2, equity_curve_ann2 = backtest_ANN2.run()

In [ ]:
analysis_ann2 = OHLCVAnalysis(asset_states_ann2, equity_curve_ann2, data, strategy=ANN2_strategy)
analysis_ann2.print_portfolio_metrics(full=True)


In [ ]:
OHLCVAnalysis.plot_multiple_portfolio_pnl([analysis_ann2,analysis_hybrid2])